In [1]:
import tkinter as tk
from tkinter import ttk
import sqlite3
from datetime import datetime

# SQLite database initialization
conn = sqlite3.connect('restaurant_billing.db')
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS bill_details (
        id INTEGER PRIMARY KEY,
        customer_name TEXT,
        phone_number TEXT,
        email TEXT,
        date TEXT,
        items TEXT,
        total_amount REAL,
        is_paid BOOLEAN
    )
''')
conn.commit()

# Dishes with their respective prices
dishes = {
    "Dish 1": 200,
    "Dish 2": 350,
    "Dish 3": 150,
    "Dish 4": 175,
    "Dish 5": 295,
    "Dish 6": 120,
    "Dish 7": 450,
    "Dish 8": 70,
    "Dish 9": 50,
    "Dish 10": 90,
    "Dish 11": 160,
    "Dish 12": 190,
    "Dish 13": 270,
    "Dish 14": 235,
    "Dish 15": 185,
    "Dish 16": 145,
    "Dish 17": 320,
    "Dish 18": 410,
    "Dish 19": 210,
    "Dish 20": 105,
}

class RestaurantBillingSystem:
    def __init__(self, root):
        self.root = root
        self.root.title("Restaurant Billing System")
        self.current_bill = []
        self.total_amount_var = tk.StringVar(value='Total Amount: Rs.0.00')  # Define as a StringVar

        self.create_home_page()

    def create_home_page(self):
        for widget in self.root.winfo_children():
            widget.destroy()

        home_label = tk.Label(self.root, text="Welcome to the Restaurant Billing System", font=('Helvetica', 16))
        home_label.pack(pady=20)

        new_bill_button = tk.Button(self.root, text='New Bill', command=self.create_new_bill_page)
        new_bill_button.pack(pady=10)

        view_bills_button = tk.Button(self.root, text='View All Bills', command=self.view_all_bills)
        view_bills_button.pack(pady=10)

    def create_new_bill_page(self):
        for widget in self.root.winfo_children():
            widget.destroy()

        # Create a canvas with scrollbar
        canvas = tk.Canvas(self.root, borderwidth=0, background="#ffffff")
        scrollbar = tk.Scrollbar(self.root, orient="vertical", command=canvas.yview)
        canvas.config(yscrollcommand=scrollbar.set)
        scrollbar.pack(side="right", fill="y")
        canvas.pack(side="left", fill="both", expand=True)

        # Create a frame to contain the widgets
        frame = tk.Frame(canvas, background="#ffffff")
        canvas.create_window((0, 0), window=frame, anchor="nw")

        # Dishes listbox with scrollbar
        dish_list_frame = tk.Frame(frame)
        dish_list_frame.pack(pady=10)

        dish_list_scrollbar = tk.Scrollbar(dish_list_frame, orient='vertical')
        dish_list_scrollbar.pack(side='right', fill='y')

        self.dish_listbox = tk.Listbox(dish_list_frame, selectmode=tk.MULTIPLE, height=10, yscrollcommand=dish_list_scrollbar.set)
        for dish in dishes:
            self.dish_listbox.insert(tk.END, f"{dish}: Rs.{dishes[dish]:.2f}")
        self.dish_listbox.pack(side='left')

        dish_list_scrollbar.config(command=self.dish_listbox.yview)

        # Quantity entry
        self.quantity_entry = tk.Entry(frame, width=5)
        self.quantity_entry.pack(pady=5)

        # Button to add selected items to the order
        add_to_order_button = tk.Button(frame, text='Add to Order', command=self.add_to_order)
        add_to_order_button.pack(pady=10)

        # Treeview with scrollbar to display order details
        tree_frame = tk.Frame(frame)
        tree_frame.pack(pady=10)

        tree_scrollbar = tk.Scrollbar(tree_frame, orient='vertical')
        tree_scrollbar.pack(side='right', fill='y')

        self.tree = ttk.Treeview(tree_frame, columns=('Dish', 'Quantity', 'Price'), show='headings', yscrollcommand=tree_scrollbar.set)
        self.tree.heading('Dish', text='Dish')
        self.tree.heading('Quantity', text='Quantity')
        self.tree.heading('Price', text='Price')
        self.tree.pack(side='left')

        tree_scrollbar.config(command=self.tree.yview)

        # Button to calculate total bill
        calculate_button = tk.Button(frame, text='Calculate Total', command=self.calculate_total)
        calculate_button.pack(pady=10)

        # Label to dynamically display total amount
        total_label = tk.Label(frame, textvariable=self.total_amount_var, font=('Helvetica', 12, 'bold'))
        total_label.pack(pady=10)

        # Entry fields for customer details
        customer_label = tk.Label(frame, text='Customer Name:')
        customer_label.pack()
        self.customer_entry = tk.Entry(frame)
        self.customer_entry.pack(pady=5)

        phone_label = tk.Label(frame, text='Phone Number:')
        phone_label.pack()
        self.phone_entry = tk.Entry(frame)
        self.phone_entry.pack(pady=5)

        email_label = tk.Label(frame, text='Email:')
        email_label.pack()
        self.email_entry = tk.Entry(frame)
        self.email_entry.pack(pady=5)

        date_label = tk.Label(frame, text='Date:')
        date_label.pack()
        self.date_entry = tk.Entry(frame)
        self.date_entry.insert(0, datetime.now().strftime('%Y-%m-%d'))
        self.date_entry.pack(pady=5)

        # Button to complete the order
        complete_order_button = tk.Button(frame, text='Complete Order', command=self.complete_order)
        complete_order_button.pack(pady=10)

        # Back button to go to the home page
        back_button = tk.Button(frame, text='Back', command=self.create_home_page)
        back_button.pack(pady=10)

        # Update the canvas scroll region
        frame.update_idletasks()
        canvas.config(scrollregion=canvas.bbox("all"))

    def add_to_order(self):
        selected_dishes = self.dish_listbox.curselection()
        quantity = self.quantity_entry.get()

        if quantity.isdigit() and selected_dishes:
            quantity = int(quantity)
            for index in selected_dishes:
                dish = list(dishes.keys())[index]
                price = dishes[dish]
                total_price = price * quantity

                # Update Treeview with order details
                self.tree.insert('', 'end', values=(dish, quantity, f"Rs.{total_price:.2f}"))

                # Update current bill
                self.current_bill.append((dish, quantity, total_price))

    def calculate_total(self):
        # Calculate total bill amount
        total_amount = sum(item[2] for item in self.current_bill)

        # Update the total amount label text dynamically
        self.total_amount_var.set(f'Total Amount: Rs.{total_amount:.2f}')

    def complete_order(self):
        customer_name = self.customer_entry.get()
        phone_number = self.phone_entry.get()
        email = self.email_entry.get()
        date = self.date_entry.get()
        is_paid = False

        # Save order details to the database
        cursor.execute('INSERT INTO bill_details (customer_name, phone_number, email, date, items, total_amount, is_paid) VALUES (?, ?, ?, ?, ?, ?, ?)',
                       (customer_name, phone_number, email, date, self.format_items(), sum(item[2] for item in self.current_bill), is_paid))
        conn.commit()

        # Display completion message
        completion_label = tk.Label(self.root, text='Order Completed!')
        completion_label.pack(pady=10)

        # Options to mark bill as paid or exit
        mark_paid_button = tk.Button(self.root, text='Mark Bill as Paid', command=self.mark_paid)
        mark_paid_button.pack(pady=5)

        exit_button = tk.Button(self.root, text='Exit', command=self.create_home_page)
        exit_button.pack(pady=5)

    def mark_paid(self):
        cursor.execute('UPDATE bill_details SET is_paid = 1 WHERE id = (SELECT MAX(id) FROM bill_details)')
        conn.commit()
        self.create_home_page()

    def view_all_bills(self):
        for widget in self.root.winfo_children():
            widget.destroy()

        # Treeview to display all bills with scrollbar
        tree_frame = tk.Frame(self.root)
        tree_frame.pack(pady=10)

        tree_scrollbar = tk.Scrollbar(tree_frame, orient='vertical')
        tree_scrollbar.pack(side='right', fill='y')

        self.tree_all_bills = ttk.Treeview(tree_frame, columns=('ID', 'Customer Name', 'Date', 'Total Amount', 'Is Paid'), show='headings', yscrollcommand=tree_scrollbar.set)
        self.tree_all_bills.heading('ID', text='ID')
        self.tree_all_bills.heading('Customer Name', text='Customer Name')
        self.tree_all_bills.heading('Date', text='Date')
        self.tree_all_bills.heading('Total Amount', text='Total Amount')
        self.tree_all_bills.heading('Is Paid', text='Is Paid')
        self.tree_all_bills.pack(side='left')

        tree_scrollbar.config(command=self.tree_all_bills.yview)

        # Fetch all billing details from the database
        cursor.execute('SELECT * FROM bill_details')
        rows = cursor.fetchall()
        for row in rows:
            formatted_items = self.format_items(row[5])
            self.tree_all_bills.insert('', 'end', values=(row[0], row[1], row[4], f'Rs.{row[6]:.2f}', 'Paid' if row[7] else 'Not Paid'))

        # Button to go back to the home page
        back_button = tk.Button(self.root, text='Back', command=self.create_home_page)
        back_button.pack(pady=10)

    def format_items(self, items=None):
        items = items or self.current_bill
        formatted_items = ""
        for item in items:
            if isinstance(item, tuple):
                formatted_items += f"{item[0]} ({item[1]}), "
            elif isinstance(item, str):
                # Parse the stored string in the database
                parts = item.split(', ')
                if len(parts) >= 2:
                    formatted_items += f"{parts[0]} ({parts[1]}), "
        return formatted_items.rstrip(', ')



if __name__ == '__main__':
    root = tk.Tk()
    app = RestaurantBillingSystem(root)
    root.mainloop()

# Close the database connection when the program exits
conn.close()
